# Bloch-Torrey Equation

## Introduction

Here we solve the Bloch-Torrey equation on a unit square, with the diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$ all given as a generic functions.
The strong form of the Bloch-Torrey equation is given by

\begin{align}
    \frac{\partial u_x}{\partial t} &= \nabla \cdot (D \nabla u_x) - R u_x + \omega u_y  \quad x \in \Omega\\
    \frac{\partial u_y}{\partial t} &= \nabla \cdot (D \nabla u_y) - R u_y - \omega u_x  \quad x \in \Omega,
\end{align}

where $\vec{u}=[u_x,u_y]$ is the transverse magnetization, and $\Omega$ the domain.

We will consider homogeneous Neumann boundary conditions such that

\begin{align}
    \nabla \vec{u}(x) \cdot \hat{n} &= \vec{0}  \quad x \in \partial \Omega\\
\end{align}

where $\partial \Omega$ denotes the boundary of $\Omega$, and $\cdot$ is a tensor contraction.

The initial condition is given generically as

\begin{equation}
    \vec{u}(x,t=0) = \vec{u}_0 (x)  \quad x \in \Omega
\end{equation}


The resulting weak form is given by
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    &= -\int_{\Omega}
    -\vec{v} \cdot \nabla \cdot ( D \, \nabla \vec{u} ) +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega \\
    &= -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega + 
    \int_{\partial\Omega} \vec{v} \cdot (D\nabla\vec{u} \cdot \hat{n}) \, d\Gamma,
\end{align}
where $\vec{v}$ is a suitable test function.

In this notebook, we will assume homogeneous Neumann boundary conditions on all boundaries by taking $D\nabla\vec{u} \cdot \hat{n} = 0$. Therefore, the final weak form is simply
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    = -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega
\end{align}

Note: in two dimensions, the cross product is simply a scalar. However, `Tensors.jl` defines the two dimensional cross product by first extending the 2D vectors into 3D. Below, we use the symbol $\boxtimes$ to denote the scalar version, which is the same as taking the third component of the vector version

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [1]:
# using Distributed
# addprocs(8; restrict = true, enable_threaded_blas = true);
# @everywhere begin
#     include("init.jl")
# end

In [2]:
include("init.jl")

┌ Info: Recompiling stale cache file /home/jon/.julia/compiled/v0.7/BlochTorreyUtils.ji for BlochTorreyUtils [top-level]
└ @ Base loading.jl:1185
┌ Info: Recompiling stale cache file /home/jon/.julia/compiled/v0.7/BlochTorreySolvers.ji for BlochTorreySolvers [top-level]
└ @ Base loading.jl:1185


**Pack circles**: Pack circles with a specified packing density $\eta$

In [3]:
btparams = BlochTorreyParameters{Float64}(
    theta = π/2,
    AxonPDensity = 0.75,
    g_ratio = 0.8,
    D_Tissue = 25, # [μm²/s]
    D_Sheath = 25, # [μm²/s]
    D_Axon = 25 # [μm²/s]
);

In [4]:
TE_typical = 10e-3
nDim = 2
dist_typical = (2π * btparams.R_mu)/4
time_typical = TE_typical/2
D_Maximal_Dephasing = dist_typical^2/(2 * nDim * time_typical)

26.105103640881353

In [5]:
Dim = 2;
Ncircles = 20;

η = btparams.AxonPDensity; # goal packing density
ϵ = 0.1 * btparams.R_mu; # overlap occurs when distance between circle edges is ≤ ϵ
α = 1e-1; # covariance penalty weight (enforces circular distribution)
β = 1e-6; # mutual distance penalty weight
λ = 1.0; # overlap penalty weight (or lagrange multiplier for constrained version)

rs = rand(radiidistribution(btparams), Ncircles);
@time initial_circles = GreedyCirclePacking.pack(rs; goaldensity = 1.0, iters = 100)
@show estimate_density(initial_circles);

  1.954869 seconds (3.43 M allocations: 169.003 MiB, 3.63% gc time)
estimate_density(initial_circles) = 0.8209027758823344


In [6]:
@show minimum_signed_edge_distance(initial_circles);

minimum_signed_edge_distance(initial_circles) = -2.220446049250313e-16


In [7]:
@show estimate_density(initial_circles);

estimate_density(initial_circles) = 0.8209027758823344


In [8]:
@time outer_circles = EnergyCirclePacking.pack(initial_circles;
    autodiff = false,
    secondorder = false,
    setcallback = false,
    goaldensity = η,
    distancescale = btparams.R_mu,
    weights = [α, β, λ],
    epsilon = ϵ
);
inner_circles = scale_shape.(outer_circles, btparams.g_ratio);

  5.243768 seconds (11.88 M allocations: 592.758 MiB, 8.58% gc time)


In [9]:
dmin = minimum_signed_edge_distance(outer_circles)
@show covariance_energy(outer_circles)
@show estimate_density(outer_circles)
@show is_any_overlapping(outer_circles)
@show (dmin, ϵ, dmin > ϵ);

covariance_energy(outer_circles) = 0.32344867913058284
estimate_density(outer_circles) = 0.7499999999999996
is_any_overlapping(outer_circles) = false
(dmin, ϵ, dmin > ϵ) = (0.03245243435217232, 0.046000000000000006, false)


**Generate mesh**: Rectangular mesh with circles possibly only partly contained or completely excluded

In [10]:
h0 = minimum(radius.(outer_circles))*(1-btparams.g_ratio); # fraction of size of minimum torus width
h_min = 1.0*h0; # minimum edge length
h_max = 5.0*h0; # maximum edge length
h_range = 10.0*h0; # distance over which h increases from h_min to h_max
h_rate = 0.6; # rate of increase of h from circle boundaries (power law; smaller = faster radial increase)

0.6

In [11]:
bdry, _ = opt_subdomain(outer_circles);

In [12]:
@time exteriorgrids, torigrids, interiorgrids, parentcircleindices = disjoint_rect_mesh_with_tori(
    bdry, inner_circles, outer_circles, h_min, h_max, h_range, h_rate;
    CIRCLESTALLITERS = 500, EXTERIORSTALLITERS = 1000, plotgrids = false, exterior_tiling = (1, 1)
);

1/20: Interior
2/20: Interior
3/20: Interior
4/20: Interior
5/20: Interior
6/20: Interior
7/20: Interior
8/20: Interior
9/20: Interior
10/20: Interior
11/20: Interior
12/20: Interior
13/20: Interior
14/20: Interior
15/20: Interior
16/20: Interior
17/20: Interior
18/20: Interior
19/20: Interior
20/20: Interior
1/20: Annular
2/20: Annular
3/20: Annular
4/20: Annular
5/20: Annular
6/20: Annular
7/20: Annular
8/20: Annular
9/20: Annular
10/20: Annular
11/20: Annular
12/20: Annular
13/20: Annular
14/20: Annular
15/20: Annular
16/20: Annular
17/20: Annular
18/20: Annular
19/20: Annular
20/20: Annular
1/1, 1/1: Exterior
 57.182816 seconds (270.95 M allocations: 8.413 GiB, 10.03% gc time)


In [13]:
simpplot(vcat(exteriorgrids[:], torigrids, interiorgrids); newfigure = true, axis = mxaxis(bdry));

In [14]:
sum(area.(exteriorgrids)) + sum(area.(torigrids)) + sum(area.(interiorgrids)) - area(bdry)

-1.7763568394002505e-15

## Diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$

These functions are defined within `doassemble!`

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [15]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions are handled by a `ConstraintHandler`. However, here we will have no need to directly enforce boundary conditions, since Neumann boundary conditions have already been applied in the derivation of the weak form.

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

In [75]:
myelinprob = MyelinProblem(btparams);

In [76]:
myelindomains = createmyelindomains(exteriorgrids[:], torigrids, interiorgrids, outer_circles, inner_circles);

In [77]:
@time map!(m -> doassemble!(m, myelinprob), myelindomains, myelindomains);

  0.785202 seconds (15.28 M allocations: 288.755 MiB, 10.53% gc time)


In [78]:
@time map!(m -> (factorize!(getdomain(m)); return m), myelindomains, myelindomains);

  0.047599 seconds (22.36 k allocations: 7.434 MiB)


In [79]:
@time fullmyelindomain = MyelinDomain(PermeableInterfaceRegion(), myelinprob, myelindomains);

  4.988116 seconds (74.51 M allocations: 2.565 GiB, 18.83% gc time)


In [80]:
@time factorize!(fullmyelindomain);

  0.009448 seconds (37 allocations: 6.313 MiB)


In [81]:
# for II in ips #[rand(1:length(ips))]
#     Se = [fullmyelindomain.domain.K[2i-1,2j-1] for i in II for j in II]
#     Se = Matrix(reshape(6 .* Se, (4,4)))
#     @assert maximum(abs, sum(Se, dims=2)) < 5*eps(Float64)
# end

In [82]:
# for i in II
#     n = getnodes(getgrid(fullmyelindomain))[i]
#     @show n
# end

In [83]:
# AllNodes = getnodes(getgrid(fullmyelindomain))
# XMatrix, YMatrix = zeros(2, length(ips)), zeros(2, length(ips))
# for j in 1:length(ips)
#     idx = ips[j]
#     p1, p2 = AllNodes[idx[1]].x, AllNodes[idx[2]].x
#     XMatrix[1,j] = p1[1]
#     YMatrix[1,j] = p1[2]
#     XMatrix[2,j] = p2[1]
#     YMatrix[2,j] = p2[2]
# end

In [84]:
# mxcall(:plot, 0, XMatrix, YMatrix, "bx-")
# mxcall(:axis, 0, "image")

### Plotting the resonance frequency map $\omega(x)$
$\omega(x)$ for each region can be easily created by accessing the `Omega` field of a `BlochTorreyProblem` object. Now, evaluate $\omega(x)$ on each node `x` and plot the resuling field map overtop of the tesselation.

In [85]:
omegavalues = omegamap.(Ref(myelinprob), myelindomains);

In [86]:
simpplot(getgrid.(myelindomains); newfigure = true, axis = mxaxis(bdry), facecol = reduce(vcat, omegavalues));

### Solution of the differential equation system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [87]:
tspan = (0.0, 320.0e-3);
dt = 10e-3;
ts = tspan[1]:dt:tspan[2]
# saveat = tspan[1]:dt:tspan[2];
# tstops = (tspan[1] .+ dt/2 .+ dt .* (1:round(Int, (tspan[2]-tspan[1])/dt)))
u0 = Vec{2}((0.0, 1.0)); # initial pi/2 pulse

In [88]:
probs = [ODEProblem(m, interpolate(u0, m), tspan) for m in [fullmyelindomain]];

In [89]:
sols = Vector{ODESolution}(undef, length(probs));

In [90]:
@time sols = map!(sols, 1:length(probs), probs) do i, prob
    print("i = $i/$(length(sols)): ")
    A = prob.p[1]
    return @time solve(prob, ExpokitExpmv(A; m = 30);
        dt = dt,
        reltol = 1e-4,
        callback = MultiSpinEchoCallback(tspan; TE = dt)
    )
end;

i = 1/1:  93.873167 seconds (527.32 k allocations: 15.701 GiB, 4.64% gc time)
 93.928095 seconds (589.30 k allocations: 15.704 GiB, 4.63% gc time)


In [91]:
Umag = reduce(vcat, norm.(reinterpret(Vec{2,Float64}, s.u[end])) for s in sols);

In [92]:
simpplot(getgrid(fullmyelindomain); newfigure = true, axis = mxaxis(bdry), facecol = Umag);

In [93]:
Signals = map([fullmyelindomain], sols) do m, s
    [integrate(s(t), m) for t in tspan[1]:dt:tspan[2]]
end;

In [94]:
Stotal = sum(Signals);

In [95]:
Tspan = tspan[2] - tspan[1]
@show btparams.R2_lp;
@show (-1/Tspan)*log(norm(Stotal[end])/norm(Stotal[1]));
@show exp(-tspan[end]*btparams.R2_lp);
@show norm(Stotal[end])/norm(Stotal[1]);

btparams.R2_lp = 15.873015873015873
(-1 / Tspan) * log(norm(Stotal[end]) / norm(Stotal[1])) = 16.8012565951162
exp(-(tspan[end]) * btparams.R2_lp) = 0.006223859418487457
norm(Stotal[end]) / norm(Stotal[1]) = 0.004624430223909732


In [96]:
exact_mwf = getmwf(outer_circles, inner_circles, bdry);
@show exact_mwf;

exact_mwf = 0.2537751490041915


In [97]:
mxcall(:addpath, 0, realpath("MATLAB/SE_corr/"));

In [98]:
MWImaps, MWIdist, MWIpart = fitmwfmodel(Stotal, NNLSRegression();
    T2Range = [8e-3, 2.0],
    spwin = [8e-3, 24.75e-3],
    mpwin = [25.25e-3, 200e-3],
    nT2 = 32,
    RefConAngle = 165.0,
    PLOTDIST = true
);
getmwf(NNLSRegression(), MWImaps, MWIdist, MWIpart)

0.23526031384011473

In [172]:
# TestStotalMagn = mwimodel(TwoPoolMagnToMagn(), tspan[1]:dt:tspan[2], modelfit.param)
# TestStotal = [Vec{2}((zero(y),y)) for y in TestStotalMagn];

In [51]:
for modeltype in (TwoPoolMagnToMagn(), ThreePoolMagnToMagn(), ThreePoolCplxToMagn(), ThreePoolCplxToCplx())
    local modelfit, errors, mwf
    println("Model: $modeltype"); flush(stdout)
    modelfit, errors = fitmwfmodel(Stotal, modeltype; TE = dt);
    mwf = getmwf(modeltype, modelfit, errors)
    println("mwf: $mwf"); flush(stdout)
    errors == nothing ? display(modelfit.param) : display([modelfit.param errors]); flush(stdout)
end

Model: TwoPoolMagnToMagn()
mwf: 0.5441897512778306


4×2 Array{Float64,2}:
  3.09771   13.783 
  2.59462   14.8841
 40.0      381.687 
  6.66667   31.9377

Model: ThreePoolMagnToMagn()
mwf: 0.8808246968249974


6-element Array{Float64,1}:
 11.210967506762957
  0.0              
  1.516840361447119
 40.0              
  6.666666666666667
  6.666666666666667

Model: ThreePoolCplxToMagn()
mwf: 0.10565155595890315


8×2 Array{Float64,2}:
  1.01505       939.651    
  5.71283         1.96669e5
  2.87967         1.95445e5
 92.9468          1.47292e5
 15.3174      41360.8      
 20.0758          2.51193e5
  1.30218         1.65867e6
 -0.00514191      2.40204e6

Model: ThreePoolCplxToCplx()
mwf: 0.5732925969044638


10-element Array{Float64,1}:
  18.874364836492983 
   0.0               
  14.04837817188795  
 333.3333333333333   
  10.336273250604892 
  10.13019548977947  
 -25.18618283265522  
  50.64733690952367  
  50.51747448309081  
  -1.5503681393765443

In [174]:
widths(bdry)

2-element Tensor{1,2,Float64,2}:
 4.597908069979724
 4.597908069979724

In [52]:
@show getmwf(outer_circles, inner_circles, bdry)
@show getmwf(Stotal, TwoPoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToCplx(); TE = dt, fitmethod = :local);

getmwf(outer_circles, inner_circles, bdry) = 0.2537751490041915
getmwf(Stotal, TwoPoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.5441897512778306
getmwf(Stotal, ThreePoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.8808246968249974
getmwf(Stotal, ThreePoolCplxToMagn(); TE=dt, fitmethod=:local) = 0.10565155595890315
getmwf(Stotal, ThreePoolCplxToCplx(); TE=dt, fitmethod=:local) = 0.5732925969044638


In [57]:
p0 = initialparams(ThreePoolCplxToCplx(), ts, Stotal)[1];
modelfit, errors = fitmwfmodel(Stotal, ThreePoolCplxToCplx(); TE = dt);

In [177]:
# mwimodel(ThreePoolCplxToCplx(), ts, modelfit.param);
# [mwimodel(ThreePoolCplxToCplx(), ts,  modelfit.param) |> x -> reinterpret(ComplexF64, x) complex.(Stotal)]
# [mwimodel(ThreePoolCplxToCplx(), ts, p0) |> x -> reinterpret(ComplexF64, x) complex.(Stotal)]

In [58]:
myelin_area = intersect_area(outer_circles, bdry) - intersect_area(inner_circles, bdry)
total_area = area(bdry)
y_biexp = @. (total_area - myelin_area) * exp(-ts*btparams.R2_lp) + myelin_area * exp(-ts*btparams.R2_sp);

In [59]:
mxcall(:figure, 0)
mxcall(:plot, 0, collect(1000.0.*ts), [norm.(Stotal) y_biexp])

In [60]:
mxcall(:legend, 0, "Simulated", "Bi-Exponential")
mxcall(:title, 0, "Signal Magnitude vs. Time")
mxcall(:xlabel, 0, "Time [ms]"); mxcall(:xlim, 0, 1000.0 .* [tspan...])
mxcall(:ylabel, 0, "S(t) Magnitude")

In [61]:
# mxcall(:figure, 0)
# mxcall(:plot, 0, collect(1000.0.*ts), reduce(hcat, map(S->norm.(S), Signals)))

In [39]:
# prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), u0, tspan, (Amap,));

In [40]:
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=100); # penelope: 17.42s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=50); # penelope: 30.09s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=10); # penelope: 103.5s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-8, norm=expmv_norm); # penelope: 53.2s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-6, norm=expmv_norm); # penelope: 44.4s

In [41]:
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-8, alg_hints=:stiff); # penelope: 90.21s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff); # penelope: 33.44s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:BCG); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 53.66s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:TFQMR); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 18.99s but low accuracy

In [42]:
#prob_Ku = ODEProblem(K_mul_u!, u0, tspan, (K,), mass_matrix=M);
#@time sol_Ku = solve(prob_Ku, Rosenbrock23(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF
#@time sol_Ku = solve(prob_Ku, Rodas4(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF

In [43]:
# @show norm(sol.u[end] - u)/maximum(abs,u);
# @show maximum(sol.u[end] - u)/maximum(abs,u);

# Testing

In [ ]:
using BlochTorreyUtilsTest

### Single Axon

In [ ]:
BlochTorreyUtilsTest.singleaxontests(
    BlochTorreyParameters{Float64}(
        ChiI = -60e-9,
        ChiA = -120e-9
    );
    PLOTOMEGA = false
);

### Multiple Axons

In [ ]:
domainsetup = BlochTorreyUtilsTest.multipleaxons();

In [ ]:
BlochTorreyUtilsTest.multipleaxontests(
    BlochTorreyParameters{Float64}(
        ChiI = -60e-9,
        ChiA = -120e-9
    ),
    domainsetup;
    PLOTOMEGA = false
);

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
# vtk_grid("bloch_torrey_equation", dh) do vtk
#     vtk_point_data(vtk, dh, u)
# end